In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import keras_nlp
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import re
import random
import string

In [4]:
text_pairs = []

def load_text(filename):
    with open(f'Texts/{filename}', encoding='utf-8') as file_book:
        text = file_book.read()
    for pair in text.split('\n\n'):
        klm, ru = pair.split('\n')
        klm = '[start] ' + klm + ' [end]'
        text_pairs.append((ru, klm))
        

def load_texts(filenames):
    [load_text(filename) for filename in filenames]

load_texts(['text1.txt', 'text2.txt', 'text3.txt', 'text4.txt'])

In [5]:
for _ in range(5):
    print(random.choice(text_pairs))

('Прячешь, что же ты глаза?', '[start] Нүүдән яhад бултулнач?  [end]')
('Доброе утро', '[start] Сән хонвт [end]')
('Как тебя зовут', '[start] Чини нерн кемб [end]')
('Вспоминаю заботливую мать', '[start] Килмҗтә ээҗм сангдна [end]')
('младший', '[start] дү [end]')


In [6]:
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples : num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples :]

print(f"{len(text_pairs)} total pairs")
print(f"{len(train_pairs)} training pairs")
print(f"{len(val_pairs)} validation pairs")
print(f"{len(test_pairs)} test pairs")

1599 total pairs
1121 training pairs
239 validation pairs
239 test pairs


In [7]:
strip_chars = string.punctuation.replace("[", "")
strip_chars = strip_chars.replace("]", "")

vocab_size = 2400
sequence_length = 20
batch_size = 16


def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, "[%s]" % re.escape(strip_chars), "")


ru_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
klm_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)

train_ru_texts = [pair[0] for pair in text_pairs]
train_klm_texts = [pair[1] for pair in text_pairs]
ru_vectorization.adapt(train_ru_texts)
klm_vectorization.adapt(train_klm_texts)

In [8]:
def format_dataset(ru, klm):
    ru = ru_vectorization(ru)
    klm = klm_vectorization(klm)
    return ({"encoder_inputs": ru, "decoder_inputs": klm[:, :-1],}, klm[:, 1:])


def make_dataset(pairs):
    ru_texts, klm_texts = zip(*pairs)
    ru_texts = list(ru_texts)
    klm_texts = list(klm_texts)
    dataset = tf.data.Dataset.from_tensor_slices((ru_texts, klm_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset)
    return dataset.shuffle(2048).prefetch(16).cache()


train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [9]:
for inputs, targets in train_ds.take(1):
    print(f'inputs["encoder_inputs"].shape: {inputs["encoder_inputs"].shape}')
    print(f'inputs["decoder_inputs"].shape: {inputs["decoder_inputs"].shape}')
    print(f"targets.shape: {targets.shape}")

inputs["encoder_inputs"].shape: (16, 20)
inputs["decoder_inputs"].shape: (16, 20)
targets.shape: (16, 20)


In [10]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, mask=None):
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
        attention_output = self.attention(
            query=inputs, value=inputs, key=inputs, attention_mask=padding_mask
        )
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)


class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)


class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, latent_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.latent_dim = latent_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim
        )
        self.dense_proj = keras.Sequential(
            [layers.Dense(latent_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)

        attention_output_1 = self.attention_1(
            query=inputs, value=inputs, key=inputs, attention_mask=causal_mask
        )
        out_1 = self.layernorm_1(inputs + attention_output_1)

        attention_output_2 = self.attention_2(
            query=out_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        out_2 = self.layernorm_2(out_1 + attention_output_2)

        proj_output = self.dense_proj(out_2)
        return self.layernorm_3(out_2 + proj_output)

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)],
            axis=0,
        )
        return tf.tile(mask, mult)

In [14]:
embed_dim = 256
latent_dim = 2048
num_heads = 8

encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="encoder_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, latent_dim, num_heads)(x)
encoder = keras.Model(encoder_inputs, encoder_outputs)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="decoder_inputs")
encoded_seq_inputs = keras.Input(shape=(None, embed_dim), name="decoder_state_inputs")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, latent_dim, num_heads)(x, encoded_seq_inputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
decoder = keras.Model([decoder_inputs, encoded_seq_inputs], decoder_outputs)

decoder_outputs = decoder([decoder_inputs, encoder_outputs])
transformer = keras.Model(
    [encoder_inputs, decoder_inputs], decoder_outputs, name="transformer"
)

In [15]:
epochs = 1

transformer.summary()
transformer.compile(
    "rmsprop", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 positional_embedding_2 (Positi  (None, None, 256)   619520      ['encoder_inputs[0][0]']         
 onalEmbedding)                                                                                   
                                                                                                  
 decoder_inputs (InputLayer)    [(None, None)]       0           []                               
                                                                                                  
 transformer_encoder_1 (Transfo  (None, None, 256)   3155456     ['positional_embedding_

In [20]:
history = transformer.fit(train_ds, epochs=10, validation_data=val_ds)

Epoch 1/10
71/71 [==============================] - 2s 27ms/step - loss: 1.1574 - accuracy: 0.4178 - val_loss: 1.2707 - val_accuracy: 0.4187
Epoch 2/10
71/71 [==============================] - 2s 27ms/step - loss: 1.1267 - accuracy: 0.4211 - val_loss: 1.2781 - val_accuracy: 0.4162
Epoch 3/10
71/71 [==============================] - 2s 28ms/step - loss: 1.0923 - accuracy: 0.4228 - val_loss: 1.2707 - val_accuracy: 0.4203
Epoch 4/10
71/71 [==============================] - 2s 27ms/step - loss: 1.0709 - accuracy: 0.4260 - val_loss: 1.2835 - val_accuracy: 0.4129
Epoch 5/10
71/71 [==============================] - 2s 27ms/step - loss: 1.0377 - accuracy: 0.4305 - val_loss: 1.3081 - val_accuracy: 0.3947
Epoch 6/10
71/71 [==============================] - 2s 27ms/step - loss: 1.0201 - accuracy: 0.4354 - val_loss: 1.2808 - val_accuracy: 0.3980
Epoch 7/10
71/71 [==============================] - 2s 27ms/step - loss: 0.9896 - accuracy: 0.4448 - val_loss: 1.2943 - val_accuracy: 0.3964
Epoch 8/10
71

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['train','validation'])
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['train','validation'])
plt.show()

In [22]:
klm_vocab = klm_vectorization.get_vocabulary()
klm_index_lookup = dict(zip(range(len(klm_vocab)), klm_vocab))
max_decoded_sentence_leuth = 20


def decode_sequence(input_sentence):
    tokenized_input_sentence = ru_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_leuth):
        tokenized_target_sentence = klm_vectorization([decoded_sentence])[:, :-1]
        predictions = transformer([tokenized_input_sentence, tokenized_target_sentence])

        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = klm_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token

        if sampled_token == "[end]":
            break
    return decoded_sentence


text_ru_texts = [pair[0] for pair in text_pairs]
for _ in range(30):
    input_sentence = random.choice(text_ru_texts)
    translated = decode_sequence(input_sentence).replace('[start]', '')
    print(input_sentence, '->', translated.replace('[end]', ''))

Сияя взошло солнце ->  [UNK] чамаhан санхларн 
внучка по мужской линии ->  [UNK] 
Внимательная, приземистая, красиво скачущая, спокойная лошадь ->  [UNK] гинәлә hалзн гинәлә йовдг аңхт мөрмб 
класть масло в чай ->  [UNK] 
младший ->  цә 
Как нож был острым чугун ->  [UNK] бичә келхлә минь өлгәд туст болад бәәв 
год ->  цә 
В мысли моей пришедшей во сне ->  [UNK] бичә келхлә минь өлгәд туст болад багтыя 
вихрем кружась ->  [UNK] чамаhан кевтәл 
Ненастный холодный ветер ->  [UNK] сангдна 
Мечту окрылённую достигнуть успев ->  [UNK] бичә хатргдҗ ирх 
Дрожит, освещая комнату ->  [UNK] бичә хатргдҗ ирх 
Впереди судьба видна ->  [UNK] ээҗ минь әмтәхн 
Дует северный ветер ->  [UNK] чамаhан үмсий чамаhан 
Что ж,ты, милая, смотришь искоса,  ->  [UNK] гинәлә hалзн гинәлә йовдг аңхт мөрмб 
Милой любовью коль хочешь стать ->  [UNK] бичә хатргдҗ ирх 
И золото им подарило ->  [UNK] бичә хатргдҗ ирх 
Веет теплом. ->  [UNK] 
дом правительства ->  [UNK] 
Печатно написанное письмо ->  [UNK] бичә хатргдҗ